## Integrate ipywidgets, ipympl, and NexMotion API to build UI for controlling NexCOBOT MiniBot
### Note:
NexMotion Library is Windows DLL, this demo use zugbruecke, a python module, to call functions in Windows DLL under customized build Linux Docker Container. 

### How to run this demo
* Select each cell by left click it (such as click on `[1]` ) and press `shift+Enter` to run the cell. Also, could press direction key (up/down) to select cells.
* In JupyterLab mode, Right Click it and select `Create New View for Output`.
* (Option) You can try [this demo](./NexMotion-Python_API/NexMotion_API_Demo.ipynb) for more detail about using NexMotion api (less ipywidget).

### Knowing issues:
* Wave your mouse cursor inside `ipympl` canvas area when the canvas (tf view) did not update/refresh. 3D tf view is build with `ipympl` (matplotlib jupyter extension) and update in the background by `threading`. The view (canvas) maynot update when the data points updated in the background.

* `zugbruecke` may cause `threading` failed when you click button to do move command. Please restart the kernel. 

## 1. Import ipywidgets for creating interactive UI
[Reference](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html)

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HBox, VBox, AppLayout, Layout

# Utilize threading to update certain widget from background
import threading
import time

keep_monitor = True
lock = threading.Lock()
rlock = threading.RLock()

def update_display(func, widge_arr, time_step=0.5):
    global keep_monitor
    while keep_monitor:
        ret_val = func()
        if len(ret_val) != len(widge_arr):
            print("element of return value and widge are not equal!\n")
            break
        for i in range(6):
            widge_arr[i].value = ret_val[i]
        time.sleep(time_step)


## 2. Import lib for MiniBOT (control robot arm)

In [2]:
%config IPCompleter.greedy=True
import sys
sys.path.append("/home/jovyan/work/NexMotion-Python_API/")
import nexmotion as nm
from nexmotion.errors import *
from nexmotion.constants import *
import pandas as pd
dll_path = None

In [ ]:
# Start Device (robot arm)
if dll_path is None:
    nmc = nm.Control()  # will use default path for dll and ini file
else:
    nmc = nm.Control(dll_path=dll_path)
ret = nmc.deviceOpenup(type_=DEVICE_TYPE_SIMULATOR) # DEVICE_TYPE_SIMULATOR for simulation
if ret != nm.errors.SUCCESS:
    print("NMC_DeviceOpenUp error, error code =", ret)

## 3. Check group state
Here are all possible group (movement) state:
Get the state of group. You could run this cell mutliple time to check the state of robot.  
Please refer to the below table for details.

|Value | State | Description | 
|------|:-------:|-------------:| 
| 0 | GROUP_DISABLE     | A group axis is disabled. | 
| 1 | GROUP_STAND_STILL | All group axes are enabled. | 
| 2 | GROUP_STOPPED     | After NMC_GroupStop() is called, the group is stopped. |
| 3 | GROUP_STOPPING    | After NMC_GroupStop() is called, the group is stopping. |
| 4 | GROUP_MOVING      | The group is moving. |
| 5 | GROUP_HOMING      | The group is homing. |
| 6 | GROUP_ERROR_STOP  | An error is occurred in a group axis. |

##### Note: if Group State == 0 (disable), please run step 4 to servo on in order to move robot arm.


In [6]:
# Check group state
ret = nmc.groupGetState()
if ret != SUCCESS:
    print( "groupGetState error, error code =", ret)
else:
    print( "Group State = ", nmc.groupState_.value)

Group State =  1


## 4. Enable all axes of group (Servo On)
MiniBot will make a sound when it is enabled.
And, group sate will be changed (could run step **3** to check again)  
Note: need to Enable before running any movement command

In [ ]:
ret = nmc.groupEnable()
if ret != SUCCESS:
    print( "NMC_DeviceEnableAll error, error code =", ret)
else:
    print( "enable all done" )

## 5. Draw Robot 3D tf View
This cell show how to use MplVisual Class, matplotlib-base, to show transformation coordination of each joint in 3D (like ROS-RViz tf view).

#### Hint: 
In JupyterLab mode, Right Click it (left `[#]`) and select `Create New View for Output`. JupyterLab will put 3D tf view on another tab. Make it easy to check when running the rest of cell.


In [7]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Create figure canvas to hold 3D tf view
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Initial MplVisual class with built-in dh parameter
nmv = nm.api.MplVisual(ax)   

# Compute tf matrix with initial joint value (0, 90, 0, 0, -90, 0)
nmv.get_tf_data()
nmv.draw_tf_view()  # Draw 3D tf view by Matplotlib
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Testing with fake joint values
nmv.update_tf_view([0, 45, 0, 0, -45, 90])

### 6. Show Actual Joint Position
This cell can be called to check actual joint values after you move or home the Minibot.  
Here, I use `threading` to get joint position and update to text-based & ipympl widget in the background.  
For controlling real MiniBOT, make sure you do homing process (or the output will not show correctly) 

**Note:**

In current demo, the tf view (`ipympl`) canvas not work well with threading, need to wave your mouse cursor inside cavas area to trigger redraw.

In [9]:
def get_jnt_position():
    jntPos = [0.] * 6
    with lock:
        ret = nmc.groupGetActualPosAcs(jntPos)
    if ret != SUCCESS:
        print( "NMC_GroupGetActualPosAcs error, error code =", ret)
    else:
        with lock:
            nmv.update_tf_view(jntPos)
            #fig.canvas.draw()        # this will make ipympl reposne faster, but will ruin canvas soon.
            #fig.canvas.flush_events()
    return jntPos


jnt_wids = [ widgets.FloatText( value=0, description='J'+ str(i+1) + ':', disabled=True)  \
           for i in range(6)]
for i in range(6):
    display(jnt_wids[i])
        
thread_jnt = threading.Thread(target=update_display, args=(get_jnt_position, jnt_wids, 0.25))
thread_jnt.start()

FloatText(value=0.0, description='J1:', disabled=True)

FloatText(value=0.0, description='J2:', disabled=True)

FloatText(value=0.0, description='J3:', disabled=True)

FloatText(value=0.0, description='J4:', disabled=True)

FloatText(value=0.0, description='J5:', disabled=True)

FloatText(value=0.0, description='J6:', disabled=True)

### 7. Show Actual Cartesian Position 
This step can be called to check actual cartesian values.

In [13]:
def get_cart_position():
    #get current cartesian value
    cartPos = [0.] * 6
    with lock:
        ret = nmc.groupGetActualPosPcs(cartPos)
    if ret != SUCCESS:
        print( "groupGetActualPosPcs error, error code =", ret)
    else:
        pass
    return cartPos

cart_list= ['x','y','z','a','b','c']
cart_wids = [ widgets.FloatText( value=0, description= cart_list[i] + ' val' + ':', disabled=True)  \
           for i in range(6)]
for i in range(6):
    display(cart_wids[i])
        
thread_cart = threading.Thread(target=update_display, args=(get_cart_position, cart_wids, 0.25))
thread_cart.start()

FloatText(value=0.0, description='x val:', disabled=True)

FloatText(value=0.0, description='y val:', disabled=True)

FloatText(value=0.0, description='z val:', disabled=True)

FloatText(value=0.0, description='a val:', disabled=True)

FloatText(value=0.0, description='b val:', disabled=True)

FloatText(value=0.0, description='c val:', disabled=True)

## 8. Execute Group PTP motion with ipywidget UI
##### Note: Please make sure all the joints are finish homing (if controlling real MiniBOT)

### 8.1. Group PTP by absoluted joint position
Note: Please make sure you know the absoluted joint value according current robot pose.

In [14]:
# set to initial position

@interact_manual(j1=widgets.BoundedFloatText(value=0., min=-150.0, max=150.0, step=1, description='J1:', disabled=False), 
                 j2=widgets.BoundedFloatText(value=90., min=0.0, max=150.0, step=1, description='J2:', disabled=False), 
                 j3=widgets.BoundedFloatText(value=0., min=-30.0, max=50.0, step=1, description='J3:', disabled=False), 
                 j4=widgets.BoundedFloatText(value=0., min=-100.0, max=100.0, step=1, description='J4:', disabled=False), 
                 j5=widgets.BoundedFloatText(value=-90., min=-100.0, max=100.0, step=1, description='J5:', disabled=False), 
                 j6=widgets.BoundedFloatText(value=0., min=-200.0, max=200.0, step=1, description='J6:', disabled=False))
def groupPTPAbs(j1, j2, j3, j4, j5, j6):
    desPos = [j1, j2, j3, j4, j5, j6]
    nmc.groupPtpAcsAll(desPos)


interactive(children=(BoundedFloatText(value=0.0, description='J1:', max=150.0, min=-150.0, step=1.0), Bounded…

### 8.2. Group PTP by relative joint position

In [ ]:
rel_list = [-5, -1, -0.1, 0, 0.1, 1, 5]
@interact_manual(j1=widgets.Dropdown(options=rel_list,value=0,description='J1 Rel(deg):',disabled=False,), 
                 j2=widgets.Dropdown(options=rel_list,value=0,description='J2 Rel(deg):',disabled=False,), 
                 j3=widgets.Dropdown(options=rel_list,value=0,description='J3 Rel(deg):',disabled=False,), 
                 j4=widgets.Dropdown(options=rel_list,value=0,description='J4 Rel(deg):',disabled=False,), 
                 j5=widgets.Dropdown(options=rel_list,value=0,description='J5 Rel(deg):',disabled=False,), 
                 j6=widgets.Dropdown(options=rel_list,value=0,description='J6 Rel(deg):',disabled=False,))
def groupPTPRel(j1, j2, j3, j4, j5, j6):
    desPos = [0.] * 6
    ret = nmc.groupGetActualPosAcs(desPos)
    #print desPos
    desPos[0] += j1; desPos[1] += j2; desPos[2] += j3
    desPos[3] += j4; desPos[4] += j5; desPos[5] += j6
    nmc.groupPtpAcsAll(desPos)


## 9. Execute Group Line motion by relative Carteisan position

In [15]:
rel_list = [-30, -10, -5, -1, 0, 1, 5, 10, 30]
@interact_manual(x_rel=widgets.Dropdown(options=rel_list,value=0,description='X Rel(mm):',disabled=False,), 
                 y_rel=widgets.Dropdown(options=rel_list,value=0,description='Y Rel(mm):',disabled=False,), 
                 z_rel=widgets.Dropdown(options=rel_list,value=0,description='Z Rel(mm):',disabled=False,), 
                 a_rel=widgets.Dropdown(options=rel_list,value=0,description='A Rel(deg):',disabled=False,), 
                 b_rel=widgets.Dropdown(options=rel_list,value=0,description='B Rel(deg):',disabled=False,), 
                 c_rel=widgets.Dropdown(options=rel_list,value=0,description='C Rel(deg):',disabled=False,))
def groupPTPRel(x_rel, y_rel, z_rel, a_rel, b_rel, c_rel):
    desPos = [0.] * 6
    nmc.groupGetActualPosPcs(desPos)
    #print Pos
    desPos[0] += x_rel; desPos[1] += y_rel; desPos[2] += z_rel
    desPos[3] += a_rel; desPos[4] += b_rel; desPos[5] += c_rel
    nmc.groupLine(desPos)

interactive(children=(Dropdown(description='X Rel(mm):', index=4, options=(-30, -10, -5, -1, 0, 1, 5, 10, 30),…

## 10. Use AppLayout to combine all widgets (WIP)

In [16]:
fig.canvas.layout.min_height = '400px'
fig.canvas.layout.min_width = '400px'
fig.canvas.header_visible=False
fig.canvas.toolbar_visible=False
fig.canvas.footer_visible=False

jnt_grp = VBox([w for w in jnt_wids])
jnt_grp.layout.width='400px'
cart_grp = VBox([w for w in cart_wids])
cart_grp.layout.width='400px'  #'auto'

Control_UI = AppLayout(header=None,
                       left_sidebar=None,
                       center=fig.canvas,
                       right_sidebar= VBox([jnt_grp, cart_grp]),
                       footer=None,
                       pane_widths=[0, '500px', '450px'],
                       pane_heights=[0, 5, '0px'])
Control_UI

AppLayout(children=(VBox(children=(VBox(children=(FloatText(value=0.003409090909090909, description='J1:', dis…

### 11. Disable all axes of group (Servo Off)
MiniBot will make a sound when it is disabled.

In [17]:
nmc.groupDisable()

0

### 12. Shutdown device

In [18]:
# Stop threads that keep asking joint & Cartesian position
keep_monitor = False
nmc.deviceShutdown()

0